In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import math

In [5]:
data = pd.read_csv('../new-data/data_Y1.csv')
data.head()


,Year,MemberID,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,...,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated,TARGET
0,Y1,210,2.0,1.0,5.0,3.0,8,4,4,2,...,0,0,0,0,0,0,0,1,0,0
1,Y1,3889,10.0,1.0,30.0,10.0,13,7,7,1,...,0,0,0,0,1,0,1,0,0,0
2,Y1,11951,3.0,1.0,0.0,0.0,6,3,3,1,...,0,0,0,0,0,0,1,0,0,1
3,Y1,14661,2.0,1.0,1.0,1.0,2,2,2,1,...,0,0,0,0,0,0,0,1,0,0
4,Y1,14778,2.0,1.0,6.0,5.0,6,5,4,2,...,0,0,0,0,0,0,1,0,0,0


In [6]:
X = data.drop(['MemberID', 'Year', 'TARGET'], axis=1)  # Remove 'MemberID', 'Year' and 'TARGET' columns, keep the remaining columns as X
y = data['TARGET']  # Target variable y
X.head()


,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,...,age_45,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated
0,2.0,1.0,5.0,3.0,8,4,4,2,3,3,...,0,0,0,0,0,0,0,0,1,0
1,10.0,1.0,30.0,10.0,13,7,7,1,3,4,...,0,0,0,0,0,1,0,1,0,0
2,3.0,1.0,0.0,0.0,6,3,3,1,2,3,...,0,0,0,0,0,0,0,1,0,0
3,2.0,1.0,1.0,1.0,2,2,2,1,2,2,...,0,0,0,0,0,0,0,0,1,0
4,2.0,1.0,6.0,5.0,6,5,4,2,2,4,...,0,0,0,0,0,0,0,1,0,0


In [7]:
from sklearn.preprocessing import  StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

20-fold cross validation

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPRegressor

# define model
model = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=7, learning_rate='adaptive',
        learning_rate_init=0.001, max_iter=3000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=1, warm_start=False)

# Define 20-fold cross validation
kf = KFold(n_splits=20, shuffle=True, random_state=42)

# Use cross_val_score for cross validation, using negative mean squared error as the scoring criterion
scores = cross_val_score(model, X, y, cv=kf,scoring='neg_mean_squared_error')
scores = -scores

# Print the score (negative mean squared  error) for each fold and the average score
print("Score per fold: ", scores)
print("Average score: ", scores.mean())
print("variance: ", scores.var())

Iteration 1, loss = 1.57024133
Iteration 2, loss = 1.26986795
Iteration 3, loss = 1.23912201
Iteration 4, loss = 1.22302665
Iteration 5, loss = 1.21371873
Iteration 6, loss = 1.20811127
Iteration 7, loss = 1.20431170
Iteration 8, loss = 1.19990895
Iteration 9, loss = 1.19902896
Iteration 10, loss = 1.19671442
Iteration 11, loss = 1.19516996
Iteration 12, loss = 1.19453730
Iteration 13, loss = 1.19268694
Iteration 14, loss = 1.19084403
Iteration 15, loss = 1.19083098
Iteration 16, loss = 1.18984878
Iteration 17, loss = 1.18863310
Iteration 18, loss = 1.18789469
Iteration 19, loss = 1.18825826
Iteration 20, loss = 1.18707633
Iteration 21, loss = 1.18619798
Iteration 22, loss = 1.18573081
Iteration 23, loss = 1.18549373
Iteration 24, loss = 1.18519651
Iteration 25, loss = 1.18475857
Iteration 26, loss = 1.18442537
Iteration 27, loss = 1.18375076
Iteration 28, loss = 1.18366759
Iteration 29, loss = 1.18331554
Iteration 30, loss = 1.18265785
Iteration 31, loss = 1.18287222
Iteration 32, los

evaluation function

In [8]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    # The root mean square logarithmic error
    pred_y = pred_y.clip(min=0)
    pred_y = pred_y.clip(max=15)
    pred_y = pred_y.round(0).astype(int)

    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    print ("RMSLE value: %.2f" % rmsle)

    from sklearn.metrics import r2_score
    r2 = r2_score(y_test,pred_y)
    print ("R2 value: %.2f" % r2)

    from sklearn.metrics import mean_squared_error
    rmse = math.sqrt(mean_squared_error(y_test,pred_y))
    print ("RMSE value: %.2f" % rmse)
    
    from sklearn.metrics import mean_absolute_error
    mae = mean_absolute_error(y_test,pred_y)
    print(f"Mean Absolute Error (MAE): {mae}")

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90051)

In [24]:
from sklearn.neural_network import MLPRegressor

model= MLPRegressor(activation='relu', alpha=0.0002, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=7, learning_rate='adaptive',
        learning_rate_init=0.001, max_iter=3000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=1, warm_start=False)

model.fit(X_train, y_train)




Iteration 1, loss = 1.60943714
Iteration 2, loss = 1.26869911
Iteration 3, loss = 1.24152034
Iteration 4, loss = 1.22945461
Iteration 5, loss = 1.22260148
Iteration 6, loss = 1.21863287
Iteration 7, loss = 1.21491739
Iteration 8, loss = 1.21150964
Iteration 9, loss = 1.20906251
Iteration 10, loss = 1.20737082
Iteration 11, loss = 1.20580958
Iteration 12, loss = 1.20386750
Iteration 13, loss = 1.20270916
Iteration 14, loss = 1.20260934
Iteration 15, loss = 1.20041296
Iteration 16, loss = 1.19939110
Iteration 17, loss = 1.19795179
Iteration 18, loss = 1.19743169
Iteration 19, loss = 1.19712361
Iteration 20, loss = 1.19588803
Iteration 21, loss = 1.19498790
Iteration 22, loss = 1.19472586
Iteration 23, loss = 1.19377962
Iteration 24, loss = 1.19297709
Iteration 25, loss = 1.19295880
Iteration 26, loss = 1.19276931
Iteration 27, loss = 1.19212511
Iteration 28, loss = 1.19160098
Iteration 29, loss = 1.19110592
Iteration 30, loss = 1.19038798
Iteration 31, loss = 1.19028991
Iteration 32, los

MLPRegressor(alpha=0.0002, hidden_layer_sizes=7, learning_rate='adaptive',
             max_iter=3000, verbose=1)

In [25]:
print('training result: ')
eval(model,X_train, y_train)
print('testing result: ')
eval(model, X_test, y_test)

training result: 
RMSLE value: 0.53
R2 value: 0.07
RMSE value: 1.56
Mean Absolute Error (MAE): 0.6503212715590125
testing result: 
RMSLE value: 0.54
R2 value: 0.04
RMSE value: 1.57
Mean Absolute Error (MAE): 0.6678064176749079


test result

In [18]:
data_test = pd.read_csv('../new-data/data_Y2.csv')
data_test.drop("MemberID", axis = 1, inplace = True)
data_test.drop("Year", axis = 1, inplace = True)

y_test = data_test['TARGET']
X_test = data_test.loc[:, data_test.columns != 'TARGET']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [19]:
eval(model, X_test, y_test)

RMSLE value: 0.53
R2 value: 0.02
RMSE value: 1.52
Mean Absolute Error (MAE): 0.6376286134247918
